In [1]:
import os
import seaborn as sns
path = '/Users/connormcdonald/Desktop/Masters/MIT807/Gartner Repository/Analysis/Figures'
import sys
sys.path.insert(1, '/Users/connormcdonald/Desktop/Masters/MIT807/Gartner Repository/Data Collection')
from configs import *
import numpy as np
import statsmodels.formula.api as smf
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from mpl_toolkits.axisartist.axislines import SubplotZero
from pylab import text
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "lualatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

from sqlalchemy import create_engine

engine = create_engine('mysql+mysqlconnector://'+user+':'+passwd+'@'+ip+':3306/'+schema1)

In [2]:
df = pd.read_csv('/Users/connormcdonald/Desktop/Masters/MIT807/Data/twitterMAU.csv')
df['period']  = pd.to_datetime(df['period'])

In [3]:
df1 = df.iloc[:38,:]

In [4]:
import matplotlib.dates as mdates


degree = 3
y = np.array(df1['mau'].to_list())
X = np.array(mdates.date2num(df1['period']))
date_index = []
idx = 0
for i in X:
    date_index.append(idx)
    idx += 1

z = np.polyfit(date_index, y, degree)
f = np.poly1d(z)

df_trend = pd.DataFrame(columns=['X', 'y','d'])
df_trend['X'] = date_index
df_trend['y'] = y
df_trend['d'] = X


z = np.polyfit(df_trend.d, df_trend.y, degree)
model = np.poly1d(z)
results = smf.ols(formula='y ~ model(X)', data=df_trend).fit()

# print(results.summary().as_latex())

In [5]:
X = mdates.date2num(df['period'])

In [6]:
x_fit = mdates.date2num(df['period'])
y_fit = [model(_x) for _x in x_fit]

In [7]:
def dformat(d):
    return d.strftime('%Y-%m')

def yformat(d):
    return d.strftime('%Y')

In [8]:
df['y_fit'] = y_fit
df['combined_mau'] = np.where(df["mau"].isnull(), df["y_fit"], df["mau"] )*1000000
df['period_formatted'] = df['period'].apply(dformat)

In [9]:
stmt = 'SELECT DATE_FORMAT(date, \'%Y-%m\') as date, COUNT(1) AS tweets FROM social.machine_learning_only group by DATE_FORMAT(date, \'%Y-%m\') ORDER BY DATE_FORMAT(date, \'%Y-%m\') ASC' 
df2 = pd.read_sql(stmt, con=engine)

df3 = pd.merge(df, df2, left_on='period_formatted', right_on='date', how='inner')

df3['tweets_p_user'] = df3['tweets']/df3['combined_mau']

In [10]:
degree = 3
y = np.array(df3['tweets_p_user'].to_list())
X = np.array(mdates.date2num(df3['date']))
date_index = []
idx = 0
for i in X:
    date_index.append(idx)
    idx += 1

z = np.polyfit(date_index, y, degree)
f = np.poly1d(z)

df_trend = pd.DataFrame(columns=['X', 'y','d'])
df_trend['X'] = date_index
df_trend['y'] = y
df_trend['d'] = X


z = np.polyfit(df_trend.d, df_trend.y, degree)
model = np.poly1d(z)
results = smf.ols(formula='y ~ model(X)', data=df_trend).fit()


X = mdates.date2num(df3['date'])

x_fit = mdates.date2num(df3['date'])
y_fit = [model(_x) for _x in x_fit]

df3['y_fit_2'] = y_fit

In [11]:
# print(results.summary().as_latex())

In [12]:
df3.head()

,period,mau,y_fit,combined_mau,period_formatted,date,tweets,tweets_p_user,y_fit_2
0,2010-03-31,30.0,5.282818,30000000.0,2010-03,2010-03,765,0.000025,0.000047
1,2010-06-30,40.0,25.106319,40000000.0,2010-06,2010-06,701,0.000018,0.000035
2,2010-09-30,49.0,44.504028,49000000.0,2010-09,2010-09,746,0.000015,0.000025
3,2010-12-31,54.0,63.256920,54000000.0,2010-12,2010-12,744,0.000014,0.000017
4,2011-03-31,68.0,80.980447,68000000.0,2011-03,2011-03,2182,0.000032,0.000010


In [13]:
#ML plot
df3['date']  = pd.to_datetime(df3['date'])
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.rcParams["figure.figsize"] = [8, 4]
plt.rcParams.update({'font.size': 12})
plt.rcParams['figure.dpi'] = 300
plt.xlabel('Time')
plt.ylabel('Monthly Tweets Per Active User')
plt.plot(df3['date'], df3['tweets_p_user'], c = '#339898', linewidth = 1)
plt.plot(df3['date'], df3['y_fit_2'], c = '#E31B23', linewidth = 1, linestyle ="--", label= 'Polynomial Trend Line (R^2 = 0.686)')
plt.legend(loc='lower right')
plt.savefig(os.path.join(path, 'ml_normalised_tweets.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()

In [14]:
df = pd.read_csv('/Users/connormcdonald/Desktop/Masters/MIT807/Data/academic_results.csv')

In [15]:
year = ['2010-12-31', '2011-12-31', '2012-12-31', '2013-12-31', '2014-12-31', '2015-12-31', '2016-12-31', '2017-12-31', '2018-12-31', '2019-12-31', '2020-12-31', '2021-12-31']
papers = [6572, 7207, 7937, 9308, 11141, 13609, 17449, 23410, 35663, 59378, 70504, 90290]
df['papers'] = papers

In [16]:
df['scopus_norm'] = df['papers']/df['scopus_results']

In [17]:
df

,period,scholar_results,scopus_results,papers,scopus_norm
0,2010,4510000,2478126,6572,0.002652
1,2011,4590000,2638921,7207,0.002731
2,2012,4670000,2775816,7937,0.002859
3,2013,4360000,2901649,9308,0.003208
4,2014,4640000,2942515,11141,0.003786
5,2015,4710000,2953613,13609,0.004608
6,2016,3040000,3055033,17449,0.005712
7,2017,3450000,3151892,23410,0.007427
8,2018,3960000,3283566,35663,0.010861
9,2019,3500000,3437602,59378,0.017273


In [18]:
import matplotlib.dates as mdates
degree = 3
y = np.array(df['scopus_norm'].to_list())
X = np.array(mdates.date2num(df['period']))
date_index = []
idx = 0
for i in X:
    date_index.append(idx)
    idx += 1

z = np.polyfit(date_index, y, degree)
f = np.poly1d(z)

df_trend = pd.DataFrame(columns=['X', 'y','d'])
df_trend['X'] = date_index
df_trend['y'] = y
df_trend['d'] = X


z = np.polyfit(df_trend.d, df_trend.y, degree)
model = np.poly1d(z)
results = smf.ols(formula='y ~ model(X)', data=df_trend).fit()


X = mdates.date2num(df['period'])

x_fit = mdates.date2num(df['period'])
y_fit = [model(_x) for _x in x_fit]

df['y_fit_2'] = y_fit

In [19]:
# print(results.summary().as_latex())

In [20]:
#ML plot
# df['period']  = pd.to_datetime(df['period'])
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.rcParams["figure.figsize"] = [8, 4]
plt.rcParams.update({'font.size': 12})
plt.rcParams['figure.dpi'] = 300
plt.xlabel('Time')
plt.ylabel('Proportion of Machine Learning Publications')
plt.plot(df['period'], df['scopus_norm'], c = '#339898', linewidth = 1)
plt.plot(df['period'], df['y_fit_2'], c = '#E31B23', linewidth = 1, linestyle ="--", label= 'Polynomial Trend Line (R^2 = 0.901)')
plt.legend(loc='upper left')
plt.savefig(os.path.join(path, 'ml_normalised_pubs.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()